In [1]:
import pandas as pd

# Summarise detection limits

The Elveovervåkingsprogrammet report includes a table showing the proportion of values for each parameter that are at or below the detection limit. This information can be derived from `concs_and_flows_rid_20_{year}.csv`.

In [2]:
# Set year of interest
year = 2019

In [3]:
# Read data
in_csv = f"../../../Results/Loads_CSVs/concs_and_flows_rid_20_{year}.csv"
df = pd.read_csv(in_csv)

# Remove unwanted cols
del df["station_id"], df["station_code"], df["station_name"], df["old_rid_group"]
del df["new_rid_group"], df["ospar_region"], df["sample_date"], df["Qs_m3/s"]

# Count non-null entries and transpose
df = (~df.isnull()).sum()

# Restructure
df = df.reset_index()
df.columns = ["name", "count"]
df["par"], df["unit"] = df["name"].str.split("_", 1).str
del df["name"]

flg_df = df.query('unit == "flag"')
smp_df = df.query('unit != "flag"')

df = pd.merge(smp_df, flg_df, how="left", on="par", suffixes=["_smps", "_dls"])

df = df[["par", "count_smps", "count_dls"]]
df.columns = ["par", "n_samples", "n_lod"]

# Prop at LOD
df["lod_pct"] = 100 * df["n_lod"] / df["n_samples"]

# Reorder to match Eva's table
par_list = [
    "pH",
    "KOND",
    "SPM",
    "TOC",
    "TOTP",
    "PO4-P",
    "TOTN",
    "NO3-N",
    "NH4-N",
    "SiO2",
    "Pb",
    "Cd",
    "Cu",
    "Zn",
    "As",
    "Hg",
    "Cr",
    "Ni",
    "Ag",
]
ord_df = pd.DataFrame({"par": par_list})

df = pd.merge(ord_df, df, how="left", on="par")

# Save
out_csv = f"../../../Results/Loads_CSVs/lod_prop_rid_20_{year}.csv"
df.to_csv(out_csv)

df

/tmp/ipykernel_673/3618824336.py:15: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  df["par"], df["unit"] = df["name"].str.split("_", 1).str


,par,n_samples,n_lod,lod_pct
0,pH,248,0,0.000000
1,KOND,248,0,0.000000
2,SPM,248,32,12.903226
3,TOC,248,0,0.000000
4,TOTP,248,1,0.403226
5,PO4-P,248,50,20.161290
6,TOTN,248,0,0.000000
7,NO3-N,248,6,2.419355
8,NH4-N,248,87,35.080645
9,SiO2,248,0,0.000000
